# Stores Locations Installation Problem 

In [9]:
from amplpy import AMPL, Environment
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
# Ask for the AMPL folder path
amplUserFolder = input("path of the AMPL folder: ")

In [11]:
# Ask for the input data file
inputFilePath = input("path of the input file (.dat): ") or "inputs/minimart-I-50.dat"

path of the input file (.dat): 


In [12]:
# Create an AMPL instance
ampl = AMPL(Environment(amplUserFolder))

# Interpret the two files
ampl.read('minimart_sol.mod')
ampl.read_data(inputFilePath)

In [13]:
# Solve the store locations installation problem 
ampl.set_option('solver', amplUserFolder+"/cplex")
ampl.solve()

CPLEX 20.1.0.0: optimal integer solution; objective 62
0 MIP simplex iterations
0 branch-and-bound nodes


In [14]:
totalcost = ampl.get_objective('cost')
# Print it
print("Objective is:", totalcost.value())

Objective is: 62.0


In [15]:
stores = ampl.get_variable('y').get_values().toDict()

# Installed stores coordinates          
storesCoordX = ampl.get_parameter('Cx').get_values().toDict()
storesCoordY = ampl.get_parameter('Cy').get_values().toDict()
storesCoords = list()

storesIndxs = list()

for i in range(1, len(stores)+1):
    if stores.get(i) == 1:
        storesCoords.append((storesCoordX.get(i), storesCoordY.get(i)))
        storesIndxs.append(i)
        
storesCoords

[(22.0, 5.0),
 (8.0, 17.0),
 (2.0, 24.0),
 (15.0, 2.0),
 (34.0, 25.0),
 (46.0, 33.0),
 (28.0, 46.0),
 (5.0, 5.0),
 (10.0, 31.0),
 (22.0, 39.0),
 (25.0, 28.0),
 (36.0, 36.0),
 (4.0, 35.0),
 (30.0, 18.0)]

# Refurbishing Routes Problem

In [16]:
distances = ampl.get_parameter('distance').get_values().toDict()
distances

{(1.0, 1.0): 0.0,
 (1.0, 2.0): 21.400934559032695,
 (1.0, 3.0): 5.0990195135927845,
 (1.0, 4.0): 7.0,
 (1.0, 5.0): 13.038404810405298,
 (1.0, 6.0): 7.280109889280518,
 (1.0, 7.0): 17.46424919657298,
 (1.0, 8.0): 11.180339887498949,
 (1.0, 9.0): 20.8806130178211,
 (1.0, 10.0): 6.4031242374328485,
 (1.0, 11.0): 32.01562118716424,
 (1.0, 12.0): 25.179356624028344,
 (1.0, 13.0): 24.758836806279895,
 (1.0, 14.0): 38.600518131237564,
 (1.0, 15.0): 25.0,
 (1.0, 16.0): 30.610455730027933,
 (1.0, 17.0): 35.510561809129406,
 (1.0, 18.0): 26.0,
 (1.0, 19.0): 26.019223662515376,
 (1.0, 20.0): 35.35533905932738,
 (1.0, 21.0): 25.0,
 (1.0, 22.0): 17.46424919657298,
 (1.0, 23.0): 27.784887978899608,
 (1.0, 24.0): 9.055385138137417,
 (1.0, 25.0): 23.430749027719962,
 (1.0, 26.0): 8.48528137423857,
 (1.0, 27.0): 27.85677655436824,
 (1.0, 28.0): 14.317821063276353,
 (1.0, 29.0): 22.360679774997898,
 (1.0, 30.0): 16.64331697709324,
 (1.0, 31.0): 20.615528128088304,
 (1.0, 32.0): 18.439088914585774,
 (1.0

In [17]:
def computeSavings():
    savings = [(distances.get((1,i))+distances.get((1,j))-distances.get((i,j)), i, j) for i in storesIndxs  for j in storesIndxs if i!=j and i>j]
    savings.sort(reverse=True)
    return savings     

In [18]:
computeSavings()

[(58.57402427381811, 42, 14),
 (49.35766965340995, 14, 12),
 (48.21801990665334, 42, 16),
 (47.00737055009098, 16, 14),
 (44.4128293880205, 42, 12),
 (42.791845831767745, 36, 16),
 (38.49465720216175, 50, 12),
 (38.04636425820821, 50, 14),
 (37.496926147247436, 42, 36),
 (35.262818936564294, 36, 14),
 (34.66577447098989, 9, 2),
 (34.10149247694973, 40, 14),
 (33.94948268621472, 16, 12),
 (33.90034963339072, 42, 40),
 (32.817705016912775, 50, 42),
 (32.78053113415827, 40, 12),
 (29.450175629768335, 40, 16),
 (28.14120226847526, 36, 12),
 (27.90455570548353, 22, 9),
 (27.5141553629909, 50, 2),
 (27.28522592956806, 50, 40),
 (27.087187798676375, 40, 36),
 (23.916676361223484, 50, 16),
 (23.256483603679833, 12, 2),
 (23.123274861098707, 14, 2),
 (21.86518375560567, 22, 2),
 (20.32645914479174, 50, 9),
 (20.306287831220413, 50, 36),
 (18.13808644232388, 43, 16),
 (17.80004450713389, 42, 2),
 (16.890233921631044, 43, 36),
 (16.235091840445385, 24, 16),
 (16.227101861496852, 12, 9),
 (16.0341

## TODO: Clarke-Wright